# Tensorflow 2.0 Basics

## I. Tensor & Simple Gradient

In [1]:
import tensorflow as tf

#### i. Tensor
Tensor is basic data type in tensorflow

A tensor has dtype and shape.

In [12]:
r0_tensor = tf.constant(5) # rank 0 tensor
print('--- r0 tensor ---')
print(type(r0_tensor))
print(r0_tensor.shape)
print(r0_tensor.numpy())

r1_tensor = tf.constant([1,5,4.0], dtype=tf.float16)
print('--- r1 tensor ---')
print(type(r1_tensor))
print(r1_tensor.shape)
print(r1_tensor.numpy())

## rank2, 3, 4, 5, and so on

--- r0 tensor ---
<class 'tensorflow.python.framework.ops.EagerTensor'>
()
5
--- r1 tensor ---
<class 'tensorflow.python.framework.ops.EagerTensor'>
(3,)
[1. 5. 4.]


In [17]:
a = tf.constant([[5,2], [6,7]], dtype=tf.float16)
b = tf.constant([[6, 1.6], [2, 4.5]], dtype = tf.float16)

print(tf.add(a,b))
print(tf.multiply(a,b))
print(tf.matmul(a,b))
print()
print(a+b)
print(a*b)
print(a@b)

tf.Tensor(
[[11.   3.6]
 [ 8.  11.5]], shape=(2, 2), dtype=float16)
tf.Tensor(
[[30.   3.2]
 [12.  31.5]], shape=(2, 2), dtype=float16)
tf.Tensor(
[[34.  17. ]
 [50.  41.1]], shape=(2, 2), dtype=float16)

tf.Tensor(
[[11.   3.6]
 [ 8.  11.5]], shape=(2, 2), dtype=float16)
tf.Tensor(
[[30.   3.2]
 [12.  31.5]], shape=(2, 2), dtype=float16)
tf.Tensor(
[[34.  17. ]
 [50.  41.1]], shape=(2, 2), dtype=float16)


#### ii. Reshape Tensor

In [23]:
asis_tensor = tf.constant([[1,2], [3, 4]], dtype=tf.float32)
print(asis_tensor)
print(asis_tensor.shape)

tobe_tensor = tf.reshape(asis_tensor, [1, 4])
print(tobe_tensor)
print(tobe_tensor.shape)

tobe_tensor2 = tf.reshape(tobe_tensor, [2, -1]) # -1 means proper shape
print(tobe_tensor2)
print(tobe_tensor2.shape)

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
(2, 2)
tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)
(1, 4)
tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
(2, 2)


#### iii. RaggdTensor, String Tensor & Sparse Tensor

#### iv. Variables

In [27]:
t1 = tf.constant([[5,12.0], [1, 2]], dtype=tf.float32)
v1 = tf.Variable(t1)

print(v1)
print(type(v1))
print(v1.shape)

# naming
v2 = tf.Variable(t1, name="my_var1")

# turn off the gradient
v3 = tf.Variable([2,5], trainable=False)
print(v3),
print(type(v3))
print(v3.shape)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 5., 12.],
       [ 1.,  2.]], dtype=float32)>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
(2, 2)
<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([2, 5])>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
(2,)


#### v. allocate Variable & Tensor

In [32]:
with tf.device('CPU:0'):
    a = tf.Variable([5,4], dtype=tf.float32)
    b = tf.Variable([1,2], dtype=tf.float32)
    c = a*b
    
print(c)  

tf.Tensor([5. 8.], shape=(2,), dtype=float32)


#### vi. How to get gradient

In [33]:
## Gradient Tape

x = tf.Variable(5.9)

with tf.GradientTape() as tape:
    y = x*3

In [34]:
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

3.0

In [40]:
## Gradient with respect to Model

layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1, 2., 5.]])

with tf.GradientTape() as tape:
    y = layer(x)
    loss = tf.reduce_mean(y**2)
    
grad = tape.gradient(loss, layer.trainable_variables)

for v, g in zip(layer.trainable_variables, grad):
    print(f'{v.name}, shape: {g.shape} gradient:{g}')
    
[v.name for v in tape.watched_variables()]

# Tensor can not be recorded in tape. 
# It means tepe can not get gradient of tensor! but valid for variable

dense_5/kernel:0, shape: (3, 2) gradient:[[0. 0.]
 [0. 0.]
 [0. 0.]]
dense_5/bias:0, shape: (2,) gradient:[0. 0.]


['dense_5/kernel:0', 'dense_5/bias:0']

In [43]:
# select watched variable to get gradient

x0 = tf.Variable(5.0)
x1 = tf.Variable(1.6)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1) # only watch x1
    y0 = tf.math.sin(x0)
    y1 = tf.nn.softplus(x1)
    y = y0 + y1
    ys = tf.reduce_sum(y)
    
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print(f"gradient for x0 : {grad['x0']}")
print(f"gradient for x1 : {grad['x1']}")

gradient for x0 : None
gradient for x1 : 0.8320183753967285


In [50]:
## persistent resource of gradient

x = tf.constant([5., 3.0])
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x
    z = y * y
print(tape.gradient(z, x).numpy())
try:
    print(tape.gradient(z, y).numpy())
except Exception as e:
    print('not work')
    print(e)

x = tf.constant([5., 3.0])
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    y = x * x
    z = y * y
print(tape.gradient(z, x).numpy())
print(tape.gradient(z, y).numpy())


[500. 108.]
not work
A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)
[500. 108.]
[50. 18.]


## Graph and tf.function